In [2]:
# качаем данные
import requests #код взят от семинариста, разрешение на использование получено)

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                
download_file_from_google_drive('15S0KQwoGPJAZ1mclzVz8OHOp1FChSwpf', 'file.zip')

In [3]:
import zipfile # вроде стандартная либа
with zipfile.ZipFile('file.zip', 'r') as zip_ref:
    zip_ref.extractall()
    

In [88]:
#!g1.1
from IPython.display import clear_output
from tqdm.auto import tqdm
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
# import sacrebleu 
import torchtext
from torchtext.vocab import build_vocab_from_iterator
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

In [89]:
#!g1.1

def yield_sentences(filename, n=None):
    with open(filename, encoding='utf-8', mode='r') as f:
        line = f.readline().split()
        i = 0
        while line and (n is None or i < n):
            yield line
            line = f.readline().split()
            i += 1
            

In [90]:
#!g1.1
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

train_path_de = "train.de-en.de"
train_path_en = "train.de-en.en"

val_path_de = "val.de-en.de"
val_path_en = "val.de-en.en"

train_small_path_de = "train_small.de-en.de"
train_small_path_en = "train_small.de-en.en"

vocab_de = build_vocab_from_iterator(yield_sentences(train_path_de), specials=special_symbols, special_first=True)
vocab_en = build_vocab_from_iterator(yield_sentences(train_path_en), specials=special_symbols, special_first=True)
vocab_en.set_default_index(UNK_IDX)
vocab_de.set_default_index(UNK_IDX)

In [91]:
#!g1.1
# код далее взят из торчовского туториала про использование трансформера

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1, 
                 activation='relu'):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout, 
                                       activation=activation)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [92]:
#!g1.1
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [93]:
#!g1.1
torch.manual_seed(0)
SRC_VOCAB_SIZE = len(vocab_de)
TGT_VOCAB_SIZE = len(vocab_en)

In [94]:
#!g1.1

EMB_SIZE = 512 * 2
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 4
NUM_DECODER_LAYERS = 4
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)


In [95]:
#!g1.1
from torch.nn.utils.rnn import pad_sequence
from typing import Iterable, List

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# function to collate data samples into batch tesors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(tensor_transform(vocab_de(src_sample)))
        tgt_batch.append(tensor_transform(vocab_en(tgt_sample)))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

In [96]:
#!g1.1
from torch.utils.data import IterableDataset

class MyIterableDataset(IterableDataset):
    def __init__(self, path_de, path_en) -> None:
        super().__init__()
        self.path_de = path_de
        self.path_en = path_en

    def yielder(self):
        for a in zip(yield_sentences(self.path_de), yield_sentences(self.path_en)):
            yield a

    def __len__(self):
        with open(self.path_de, encoding='utf-8', mode='r') as f:
            return len(f.readlines())

    def __iter__(self):
        return self.yielder()
    


In [97]:
#!g1.1
class SmoothingCrossEntropyLoss(nn.Module):
    def __init__(self, label_smoothing=0.0, ignore_index=-100):
        super(SmoothingCrossEntropyLoss, self).__init__()
        assert 0 <= label_smoothing <= 0.5
        self.smoothing = label_smoothing
        self.ignore_index = ignore_index

    def forward(self, y_pred, y_true):
        pred = y_pred.log_softmax(dim=-1)
        
        with torch.no_grad():
            ind = (y_true == self.ignore_index)
            not_ignored_num = len(y_true) - torch.sum(ind).item()
            w = torch.full(pred.shape, self.smoothing / (TGT_VOCAB_SIZE - 1)).to(DEVICE)
            w.scatter_(1, y_true.data.unsqueeze(1), 1 - self.smoothing)
            w[ind, :] = 0.0
        return torch.mean(torch.sum(-w * pred, dim=1)) * len(y_true) / not_ignored_num 

In [98]:
#!g1.1
from torch.utils.data import DataLoader

def train_epoch(model, optimizer, scheduler=None, swa_transformer=None, return_loader=False, update_grad_freq=1, train_small=False, smoothing=None):
    model.train()
    losses = 0
    if train_small:
        train_iter = MyIterableDataset(train_small_path_de, train_small_path_en)
    else:
        train_iter = MyIterableDataset(train_path_de, train_path_en)
    train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)
    if smoothing is not None:
        loss_fn = SmoothingCrossEntropyLoss(smoothing, ignore_index=PAD_IDX)

    optimizer.zero_grad()
    for batch_num, (src, tgt) in enumerate(tqdm(train_dataloader)):
        
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()
        
        if (batch_num + 1) % update_grad_freq == 0:
            optimizer.step()
            optimizer.zero_grad()
            if scheduler is not None:
                scheduler.step()
        losses += loss.item()
        
        
        
        
#     if scheduler is not None and swa_transformer is None:
#         scheduler.step(loss.item())
        
    
    if return_loader:
        return losses / len(train_dataloader), train_dataloader
    return losses / len(train_dataloader)


def evaluate(model):
    model.eval()
    losses = 0

    # val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    val_dataloader = DataLoader(MyIterableDataset(val_path_de, val_path_en), batch_size=BATCH_SIZE, collate_fn=collate_fn)
    loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)
    
    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(val_dataloader)

In [99]:
#!g1.1
# FINAL education

from torch.optim.lr_scheduler import StepLR, ExponentialLR, ReduceLROnPlateau, CyclicLR, OneCycleLR
from timeit import default_timer as timer
NUM_EPOCHS = 15
max_lr = 0.0001
xbs=4
loader_len = 1531

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM, activation='gelu')
for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
transformer = transformer.to(DEVICE)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0, betas=(0.9, 0.98), eps=1e-9)
scheduler = OneCycleLR(optimizer, max_lr=max_lr, epochs=NUM_EPOCHS, steps_per_epoch=loader_len // xbs + 1, pct_start=0.15, anneal_strategy='cos')
scheduler_pl = ReduceLROnPlateau(optimizer, patience=3, verbose=True, factor=0.5)

SOTA_val_results = []
lrs = []

sched_epoch = 5

for epoch in range(1, NUM_EPOCHS+1):
    lrs.append(scheduler.get_last_lr()) 
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer, scheduler=scheduler, update_grad_freq=xbs, train_small=False, smoothing=0.1) 
    end_time = timer()
    val_loss = evaluate(transformer)
    if epoch > sched_epoch:
        scheduler_pl.step(val_loss)
    SOTA_val_results.append(val_loss)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))



Epoch: 1, Train loss: 8.642, Val loss: 6.087, Epoch time = 1978.743s

Epoch: 2, Train loss: 6.017, Val loss: 4.549, Epoch time = 1982.179s

Epoch: 3, Train loss: 5.000, Val loss: 3.654, Epoch time = 1986.753s

Epoch: 4, Train loss: 4.408, Val loss: 3.089, Epoch time = 1984.015s

Epoch: 5, Train loss: 4.016, Val loss: 2.833, Epoch time = 1981.933s

Epoch: 6, Train loss: 3.744, Val loss: 2.673, Epoch time = 1984.623s

Epoch: 7, Train loss: 3.548, Val loss: 2.593, Epoch time = 1984.424s

Epoch: 8, Train loss: 3.397, Val loss: 2.530, Epoch time = 1980.055s

Epoch: 9, Train loss: 3.279, Val loss: 2.493, Epoch time = 1977.020s

Epoch: 10, Train loss: 3.187, Val loss: 2.478, Epoch time = 1981.791s

Epoch: 11, Train loss: 3.118, Val loss: 2.440, Epoch time = 1987.672s

Epoch: 12, Train loss: 3.066, Val loss: 2.445, Epoch time = 1993.397s

Epoch: 13, Train loss: 3.030, Val loss: 2.447, Epoch time = 1983.114s

Epoch: 14, Train loss: 3.009, Val loss: 2.447, Epoch time = 1979.868s



Epoch: 15, Train loss: 3.000, Val loss: 2.449, Epoch time = 1981.118s


In [107]:
# #!g1.1
# # дообучение (решил не делать, не верю, что даст что-то)

# from torch.optim.lr_scheduler import StepLR, ExponentialLR, ReduceLROnPlateau, CyclicLR, OneCycleLR
# from timeit import default_timer as timer

# # # load model
# # transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM, activation='gelu')
# # checkpoint = torch.load('model_final.pt')

# # transformer.load_state_dict(checkpoint['model_state_dict'])
# # transformer = transformer.to(DEVICE)

# NUM_EPOCHS = 3
# max_lr = 0.0001
# xbs=8
# loader_len = 1531

# optimizer = torch.optim.SGD(transformer.parameters(), lr=0.0, momentum=0.9)
# scheduler = OneCycleLR(optimizer, max_lr=max_lr, epochs=NUM_EPOCHS, steps_per_epoch=loader_len // xbs + 1, pct_start=0.5, anneal_strategy='linear')
# scheduler_pl = ReduceLROnPlateau(optimizer, patience=3, verbose=True, factor=0.5)

# SOTA_val_results_2 = []
# lrs = []

# sched_epoch = 0

# for epoch in range(1, NUM_EPOCHS+1):
#     lrs.append(scheduler.get_last_lr()) 
#     start_time = timer()
#     train_loss = train_epoch(transformer, optimizer, scheduler=scheduler, update_grad_freq=xbs, train_small=False, smoothing=0.01) 
#     end_time = timer()
#     val_loss = evaluate(transformer)
#     if epoch > sched_epoch:
#         scheduler_pl.step(val_loss)
#     SOTA_val_results_2.append(val_loss)
#     print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))


NotImplementedError: 

In [101]:
#!g1.1
# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = tensor_transform(vocab_de(src_sentence.split())).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_en.lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")
translate(transformer, 'ich hasse kinder !')

' i hate kids ! '

In [102]:
#!g1.1
my_file_path_de = "test1.de-en.de"
with open(my_file_path_de, encoding='utf-8', mode='r') as f_opened, open('test1.de-en.en', encoding='utf-8', mode='w') as f_answer:
    line = f_opened.readline()
    while line:
        f_answer.write(translate(transformer, line).strip() + '\n')
        line = f_opened.readline()

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:859: UserWarning: The following variables cannot be serialized: f_answer, f_opened
  warnings.warn(message)


In [103]:
#!g1.1
# заархиватор
from zipfile import ZipFile
zipped_file = ZipFile('answer1.zip', 'w')
zipped_file.write('test1.de-en.en')
zipped_file.close()

In [ ]:
#!g1.1


In [ ]:
#!g1.1
